In [3]:
import pandas as pd
import numpy as np
import requests
import time
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import *
import calendar
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [2]:
df=pd.read_csv('scooter_weather.csv')
df.head()

C:\Users\Amro_Elhag\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Destination Cell ID,Council District (End),Council District (Start),Day of Week,Device ID,End Latitude,End Longitude,End Time,Hour,ID,...,date,sunrise,sunset,hour,temperature,heat_index,feels_like,precipitation,weather_description,hour_formatted
0,006182,8.0,8.0,2.0,4f49b46d-9fdf-434b-a2c7-77bc8253a371,30.219000,-97.871000,04/03/2018 07:00:00 PM,18.0,dce23579-adac-49b1-962e-5c52b72c2d1c,...,2018-04-03,07:18 AM,07:51 PM,1800,71,73,70,0.0,Partly cloudy,18.0
1,014548,9.0,9.0,3.0,4f49b46d-9fdf-434b-a2c7-77bc8253a371,30.250895,-97.702772,04/04/2018 10:30:00 AM,10.0,9e790f62-e5c6-4266-a517-be4bd7f98e3a,...,2018-04-04,07:17 AM,07:52 PM,1000,64,64,63,0.1,Moderate rain at times,10.0
2,006182,8.0,9.0,3.0,4f49b46d-9fdf-434b-a2c7-77bc8253a371,30.219000,-97.871000,04/04/2018 04:30:00 PM,15.0,efad2522-a9d0-41d9-bdfc-878cab9e27a3,...,2018-04-04,07:17 AM,07:52 PM,1500,73,73,73,0.0,Patchy rain possible,15.0
3,014548,9.0,9.0,3.0,5284f1ad-f179-4073-82b1-699d5cec4a45,30.250895,-97.702772,04/04/2018 05:30:00 PM,17.0,444097b3-836f-4fb2-9dc0-596ce335c406,...,2018-04-04,07:17 AM,07:52 PM,1700,70,71,70,0.0,Sunny,17.0
4,014385,9.0,9.0,4.0,bf4855b1-823b-40c6-a75c-431de50d314a,30.232776,-97.688667,04/05/2018 05:00:00 AM,5.0,87512f5d-1ad3-4c9c-919c-4b86cc83628e,...,2018-04-05,07:16 AM,07:53 PM,500,54,54,53,0.0,Partly cloudy,5.0


In [12]:
df.rename(columns={'hour':'hours'}, inplace=True)
df.head()

,Destination Cell ID,Council District (End),Council District (Start),Day of Week,Device ID,End Latitude,End Longitude,End Time,Hour,ID,...,date,sunrise,sunset,hours,temperature,heat_index,feels_like,precipitation,weather_description,hour_formatted
0,006182,8.0,8.0,2.0,4f49b46d-9fdf-434b-a2c7-77bc8253a371,30.219000,-97.871000,04/03/2018 07:00:00 PM,18.0,dce23579-adac-49b1-962e-5c52b72c2d1c,...,2018-04-03,07:18 AM,07:51 PM,1800,71,73,70,0.0,Partly cloudy,18.0
1,014548,9.0,9.0,3.0,4f49b46d-9fdf-434b-a2c7-77bc8253a371,30.250895,-97.702772,04/04/2018 10:30:00 AM,10.0,9e790f62-e5c6-4266-a517-be4bd7f98e3a,...,2018-04-04,07:17 AM,07:52 PM,1000,64,64,63,0.1,Moderate rain at times,10.0
2,006182,8.0,9.0,3.0,4f49b46d-9fdf-434b-a2c7-77bc8253a371,30.219000,-97.871000,04/04/2018 04:30:00 PM,15.0,efad2522-a9d0-41d9-bdfc-878cab9e27a3,...,2018-04-04,07:17 AM,07:52 PM,1500,73,73,73,0.0,Patchy rain possible,15.0
3,014548,9.0,9.0,3.0,5284f1ad-f179-4073-82b1-699d5cec4a45,30.250895,-97.702772,04/04/2018 05:30:00 PM,17.0,444097b3-836f-4fb2-9dc0-596ce335c406,...,2018-04-04,07:17 AM,07:52 PM,1700,70,71,70,0.0,Sunny,17.0
4,014385,9.0,9.0,4.0,bf4855b1-823b-40c6-a75c-431de50d314a,30.232776,-97.688667,04/05/2018 05:00:00 AM,5.0,87512f5d-1ad3-4c9c-919c-4b86cc83628e,...,2018-04-05,07:16 AM,07:53 PM,500,54,54,53,0.0,Partly cloudy,5.0


In [13]:
connection_string = "root:easypass2@@localhost/scooter"
engine = create_engine(f'mysql://{connection_string}')

In [14]:
df.to_sql(name='scooter_merged', con=engine, if_exists='replace', index=False)

In [15]:
weather=pd.read_csv('weather_csv.csv')

In [16]:
weather.head()

,date,sunrise,sunset,hour,temperature,heat_index,feels_like,precipitation,weather_description
0,2018-04-03,07:18 AM,07:51 PM,0,61,63,62,0.0,Partly cloudy
1,2018-04-03,07:18 AM,07:51 PM,100,61,62,61,0.0,Partly cloudy
2,2018-04-03,07:18 AM,07:51 PM,200,60,61,60,0.2,Moderate rain at times
3,2018-04-03,07:18 AM,07:51 PM,300,59,60,59,0.3,Moderate rain at times
4,2018-04-03,07:18 AM,07:51 PM,400,59,60,59,0.1,Moderate rain at times


In [20]:
weather.to_sql(name='weather_table', con=engine, if_exists='replace', index=False)